# Creating a folder

In [1]:
ls -l

total 68
lrwxrwxrwx   1 root root       7 Aug  9 16:52 bin -> usr/bin/
drwxr-xr-x   4 root root    4096 Aug 13 12:05 boot/
-rw-r--r--   1 root root     646 May 28  2021 copyright
drwxr-xr-x  14 root root    2800 Aug 21 05:24 dev/
drwxr-xr-x 102 root root    4096 Aug 21 05:25 etc/
drwxrwxr-x   7 root hadoop  4096 Aug 21 05:24 hadoop/
drwxr-xr-x   4 root root    4096 Aug 21 05:31 home/
lrwxrwxrwx   1 root root       7 Aug  9 16:52 lib -> usr/lib/
lrwxrwxrwx   1 root root       9 Aug  9 16:52 lib32 -> usr/lib32/
lrwxrwxrwx   1 root root       9 Aug  9 16:52 lib64 -> usr/lib64/
lrwxrwxrwx   1 root root      10 Aug  9 16:52 libx32 -> usr/libx32/
drwx------   2 root root   16384 Aug  9 16:52 lost+found/
drwxr-xr-x   2 root root    4096 Aug  9 16:52 media/
drwxr-xr-x   2 root root    4096 Aug  9 16:52 mnt/
drwxr-xr-x   9 root root    4096 Aug 21 05:24 opt/
dr-xr-xr-x 186 root root       0 Aug 21 05:23 proc/
drwx------   8 root root    4096 Aug 21 05:24 root/
drwxr-xr-x  32 root root     900 A

In [2]:
cd home

/home


In [3]:
ls -l

total 8
drwxr-xr-x 2 dataproc dataproc 4096 Aug 13 12:33 dataproc/
drwxr-xr-x 2 root     root     4096 Aug 21 05:31 tanat/


In [4]:
mkdir tanat

mkdir: cannot create directory ‘tanat’: File exists


In [5]:
ls -l

total 8
drwxr-xr-x 2 dataproc dataproc 4096 Aug 13 12:33 dataproc/
drwxr-xr-x 2 root     root     4096 Aug 21 05:31 tanat/


In [6]:
cd tanat/

/home/tanat


In [7]:
ls

2008.csv


# Flights Analytics with Spark!

In [8]:
!wget https://s3.amazonaws.com/imcbucket/data/flights/2008.csv

--2023-08-21 04:18:29--  https://s3.amazonaws.com/imcbucket/data/flights/2008.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.196.32, 52.216.208.248, 52.217.64.6, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.196.32|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 689413344 (657M) [binary/octet-stream]
Saving to: ‘2008.csv’

2008.csv            100%[===================>] 657.48M  94.5MB/s    in 7.2s    

2023-08-21 04:18:36 (91.6 MB/s) - ‘2008.csv’ saved [689413344/689413344]



In [9]:
ls

2008.csv


In [8]:
!hdfs dfs -ls /user/root/

Found 2 items
drwxr-xr-x   - root hadoop          0 2023-08-21 05:40 /user/root/.sparkStaging
drwxr-xr-x   - root hadoop          0 2023-08-21 05:32 /user/root/input


In [9]:
# Create RDD of 2008.csv
airline_rdd = sc.textFile("hdfs:///user/root/input/2008.csv")
airline_rdd.take(2)

['Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay',
 '2008,1,3,4,2003,1955,2211,2225,WN,335,N712SW,128,150,116,-14,8,IAD,TPA,810,4,8,0,,0,NA,NA,NA,NA,NA']

In [10]:
#remove the header, since we cannot alter the content of a RDD, so we will have to transform
# in this case, use filter, filter the header out
header_text = airline_rdd.first()
airline_noheader_rdd = airline_rdd.filter(lambda row: row != header_text)
airline_noheader_rdd.take(2)

['2008,1,3,4,2003,1955,2211,2225,WN,335,N712SW,128,150,116,-14,8,IAD,TPA,810,4,8,0,,0,NA,NA,NA,NA,NA',
 '2008,1,3,4,754,735,1002,1000,WN,3231,N772SW,128,145,113,2,19,IAD,TPA,810,5,10,0,,0,NA,NA,NA,NA,NA']

In [11]:
#Create key,value in Dictionary format
fieldname_list = header_text.split(',')

In [13]:
def make_row(row):
    fieldvalues_list = row.split(',')
    # zip - convert key,value to tuple
    # dict - convert tuple to dictionary
    d = dict(zip(fieldname_list,fieldvalues_list))
    return d
airline_rows_rdd = airline_noheader_rdd.map(make_row)
airline_rows_rdd.take(1)

[{'Year': '2008',
  'Month': '1',
  'DayofMonth': '3',
  'DayOfWeek': '4',
  'DepTime': '2003',
  'CRSDepTime': '1955',
  'ArrTime': '2211',
  'CRSArrTime': '2225',
  'UniqueCarrier': 'WN',
  'FlightNum': '335',
  'TailNum': 'N712SW',
  'ActualElapsedTime': '128',
  'CRSElapsedTime': '150',
  'AirTime': '116',
  'ArrDelay': '-14',
  'DepDelay': '8',
  'Origin': 'IAD',
  'Dest': 'TPA',
  'Distance': '810',
  'TaxiIn': '4',
  'TaxiOut': '8',
  'Cancelled': '0',
  'CancellationCode': '',
  'Diverted': '0',
  'CarrierDelay': 'NA',
  'WeatherDelay': 'NA',
  'NASDelay': 'NA',
  'SecurityDelay': 'NA',
  'LateAircraftDelay': 'NA'}]

In [14]:
raw_df = airline_rows_rdd.toDF()
raw_df.registerTempTable('flight')
raw_df.select(['ArrDelay','DepDelay']).show(10)

/usr/lib/spark/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)
23/08/21 05:45:42 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------+--------+
|ArrDelay|DepDelay|
+--------+--------+
|     -14|       8|
|       2|      19|
|      14|       8|
|      -6|      -4|
|      34|      34|
|      11|      25|
|      57|      67|
|     -18|      -1|
|       2|       2|
|     -16|       0|
+--------+--------+
only showing top 10 rows



In [15]:
raw_df.printSchema()

root
 |-- ActualElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: string (nullable = true)
 |-- CRSDepTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- CarrierDelay: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- Diverted: string (nullable = true)
 |-- FlightNum: string (nullable = true)
 |-- LateAircraftDelay: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- NASDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- SecurityDelay: string (nullable = true)
 |-- TailNum: string (nullable = true

In [16]:
results = sqlContext.sql('select * from flight limit 10')
results.show()

+-----------------+-------+--------+-------+----------+----------+--------------+----------------+---------+------------+---------+----------+--------+-------+----+--------+--------+---------+-----------------+-----+--------+------+-------------+-------+------+-------+-------------+------------+----+
|ActualElapsedTime|AirTime|ArrDelay|ArrTime|CRSArrTime|CRSDepTime|CRSElapsedTime|CancellationCode|Cancelled|CarrierDelay|DayOfWeek|DayofMonth|DepDelay|DepTime|Dest|Distance|Diverted|FlightNum|LateAircraftDelay|Month|NASDelay|Origin|SecurityDelay|TailNum|TaxiIn|TaxiOut|UniqueCarrier|WeatherDelay|Year|
+-----------------+-------+--------+-------+----------+----------+--------------+----------------+---------+------------+---------+----------+--------+-------+----+--------+--------+---------+-----------------+-----+--------+------+-------------+-------+------+-------+-------------+------------+----+
|              128|    116|     -14|   2211|      2225|      1955|           150|             

In [19]:
raw_df.select(['ArrDelay','DepDelay']).describe().show()

+-------+-----------------+------------------+
|summary|         ArrDelay|          DepDelay|
+-------+-----------------+------------------+
|  count|          7009728|           7009728|
|   mean| 8.16845238729114| 9.972570088930182|
| stddev|38.50193694882867|35.311270777552785|
|    min|               -1|                -1|
|    max|               NA|                NA|
+-------+-----------------+------------------+



In [18]:
results = sqlContext.sql("""
    select avg(ArrDelay) AS delay, UniqueCarrier 
    from flight 
    group by UniqueCarrier
    order by delay ASC 
    limit 10""")

results.show()

+------------------+-------------+
|             delay|UniqueCarrier|
+------------------+-------------+
|-2.888673890608875|           AQ|
|1.2644089394236424|           HA|
|2.8481100056939694|           US|
| 4.111134703250211|           9E|
| 4.804346362093581|           AS|
|  5.17967817300539|           WN|
| 6.108246666107523|           F9|
| 6.598884736868734|           OO|
| 7.368539129929383|           NW|
| 7.855163154883384|           DL|
+------------------+-------------+



## Best Airline (from Arrdelay)

In [21]:
bestAirline = sqlContext.sql("""
    SELECT UniqueCarrier, AVG(ArrDelay) AS AvgArrDelay
    FROM flight
    WHERE ArrDelay IS NOT NULL
    GROUP BY UniqueCarrier
    ORDER BY AvgArrDelay ASC
    LIMIT 10
""")

bestAirline.show()

+-------------+------------------+
|UniqueCarrier|       AvgArrDelay|
+-------------+------------------+
|           AQ|-2.888673890608875|
|           HA|1.2644089394236424|
|           US|2.8481100056939694|
|           9E| 4.111134703250211|
|           AS| 4.804346362093581|
|           WN|  5.17967817300539|
|           F9| 6.108246666107523|
|           OO| 6.598884736868734|
|           NW| 7.368539129929383|
|           DL| 7.855163154883384|
+-------------+------------------+



# Worst airline (from ArrDelay)

In [23]:
worstAirline = sqlContext.sql("""
    SELECT UniqueCarrier, AVG(ArrDelay) AS AvgArrDelay
    FROM flight
    WHERE ArrDelay IS NOT NULL
    GROUP BY UniqueCarrier
    ORDER BY AvgArrDelay DESC
    LIMIT 10
""")

worstAirline.show()

+-------------+------------------+
|UniqueCarrier|       AvgArrDelay|
+-------------+------------------+
|           AA|12.607194035713981|
|           OH|11.817467683998007|
|           YV|11.775181433600808|
|           UA|11.291322186680183|
|           B6| 11.08418439051813|
|           CO|  10.9790372802913|
|           XE|10.635404800035221|
|           EV|10.208002415713782|
|           MQ| 9.890667945776498|
|           FL|  9.09137538507922|
+-------------+------------------+



# Find airlines what flew the longest accumulated distance.

In [24]:
LongestAccDIstance = sqlContext.sql("""
    SELECT UniqueCarrier, SUM(Distance) AS TotalDistance
    FROM flight
    GROUP BY UniqueCarrier
    ORDER BY TotalDistance DESC
    LIMIT 1
""")

LongestAccDIstance.show()

+-------------+-------------+
|UniqueCarrier|TotalDistance|
+-------------+-------------+
|           WN| 7.63307895E8|
+-------------+-------------+

